In [ ]:
from web3 import Web3, HTTPProvider
import binascii
import os
PRIVATE_KEY = binascii.a2b_hex(os.environ.get("PRIVATE_KEY"))
INFURA_API_KEY = os.environ['INFURA_API_KEY']
WEB3_RPC_URL = "https://rinkeby.infura.io/" + INFURA_API_KEY
NETWORK_ID = 4

web3_provider = HTTPProvider(WEB3_RPC_URL)
w3 = Web3(web3_provider)
acct = w3.eth.account.privateKeyToAccount(PRIVATE_KEY)
MY_ADDRESS = acct.address.lower()
MY_ADDRESS

In [ ]:
import json
from zero_ex.order_utils import asset_data_utils as adu

ADDRESS_BOOK = {
    4: {
      'OptionsRegistry': Web3.toChecksumAddress('0xa5b14070af70f56fc0b3216045e53d3224bb0172'),
    },
  };

option_registry_interface = json.load(open("../artifacts/OptionsRegistry.json"))

option_registry = w3.eth.contract(
    address=ADDRESS_BOOK[NETWORK_ID]["OptionsRegistry"],
    abi=option_registry_interface["abi"],
#     bytecode=option_registry_interface["bytecode"],
)
markets = option_registry.functions.getMarkets().call()
for market_address in markets:
    market_info = option_registry.functions.getMarket(market_address).call()
    print(
        "id: {5} | topic:{0}\n\tstrike: {1}\n\texpiry: {2}\n\tlong_token: '{3}'\n\tshort_token: '{4}'\n".format(
        *market_info))

In [ ]:
VETH_TOKEN = "0xc4abc01578139e2105d9c9eba0b0aa6f6a60d082"
VETH_ASSET_DATA = adu.encode_erc20_asset_data(VETH_TOKEN)
LONG_TOKEN = "0x358b48569a4a4ef6310c1f1d8e50be9d068a50c6"
LONG_ASSET_DATA = adu.encode_erc20_asset_data(LONG_TOKEN)
SHORT_TOKEN = "0x4a1f5f67c1cf90f176496aa548382c78921ae9f1"
SHORT_ASSET_DATA = adu.encode_erc20_asset_data(SHORT_TOKEN)
print(VETH_TOKEN, VETH_ASSET_DATA, LONG_TOKEN, LONG_ASSET_DATA, SHORT_TOKEN, SHORT_ASSET_DATA)
full_asset_set_data = {
    "LONG": LONG_ASSET_DATA,
    "SHORT": SHORT_ASSET_DATA,
}

In [ ]:
import requests
import zero_ex.order_utils as ou
from zero_ex.json_schemas import assert_valid

def get_orderbook(
    base_asset_data,
    quote_asset_data,
    full_set_asset_data=None,
    network_id=NETWORK_ID,
    page=1,
    per_page=20
):
    base_url = "http://localhost:3000/v2/"
    params = dict(
        baseAssetData=base_asset_data,
        quoteAssetData=quote_asset_data,
        networkId=network_id,
        page=1,
        per_page=20,
    )
    if full_set_asset_data:
        params["fullSetAssetData"] = json.dumps(full_set_asset_data)
    response = requests.get(
        "{}{}".format(base_url, "orderbook"),
        params=params
    )
    return response.json()

In [ ]:
import time
from pydex_app.utils import to_base_unit_amount
from decimal import Decimal

def make_signed_order(
    asset,
    qty,
    price,
    side,
    maker_address=MY_ADDRESS,
    expiration_time_secs=600,
    maker_fee="0",
    taker_fee="0",
    salt=None,
    taker_address=ou._Constants.null_address,
    fee_recipient_address=ou._Constants.null_address,
    sender_address=ou._Constants.null_address,
    exchange_address="0xbce0b5f6eb618c565c3e5f5cd69652bbc279f44e", # Rinkeby Exchange Address
    w3_instance=w3,
    private_key=PRIVATE_KEY,

):
    """ Convenience function for making valid orders to buy or sell
    SHORT or LONG assets against VETH.
    
    Keyword Arguments:
    asset -- str from {'LONG', 'SHORT'} to index into `full_asset_set_data`
    qty -- how much of the ticker asset you want to buy against VETH
    price -- Always in units of LONG or SHORT asset per VETH
    side -- str from {'BUY', 'SELL'}
    maker_address -- your address (defaults to MY_ADDRESS)
    """
    OPTION_ASSET_DATA = full_asset_set_data[asset]
    if side == 'BUY':
        maker_asset_data = VETH_ASSET_DATA
        taker_asset_data = OPTION_ASSET_DATA
        maker_amount = to_base_unit_amount(qty * price)
        taker_amount = to_base_unit_amount(qty)
    elif side == 'SELL':
        maker_asset_data = OPTION_ASSET_DATA
        taker_asset_data = VETH_ASSET_DATA
        maker_amount = to_base_unit_amount(qty)
        taker_amount = to_base_unit_amount(qty * price)
    else:
        raise Exception("side must be one of {'BUY', 'SELL'}")
    
    if not salt:
        # TODO(CM): Is it reasonable to pick the salt like this?
        import random
        salt = "{:.0f}".format(Decimal(random.uniform(0, 9223372036854775807)))
    if not isinstance(maker_fee, str):
        maker_fee = to_base_unit_amount(maker_fee)
    if not isinstance(taker_fee, str):
        taker_fee = to_base_unit_amount(taker_fee)
    expiration_time_secs = int(time.time() + expiration_time_secs)
    
    order = SignedOrder()
    order.maker_address = maker_address
    order.taker_address = taker_address
    order.fee_recipient_address = fee_recipient_address
    order.sender_address = sender_address
    order.maker_asset_amount = maker_amount
    order.taker_asset_amount = taker_amount
    order.maker_fee = "{:.0f}".format(Decimal(maker_fee))
    order.taker_fee = "{:.0f}".format(Decimal(taker_fee))
    order.expiration_time_secs = expiration_time_secs
    order.salt = salt
    order.maker_asset_data = maker_asset_data
    order.taker_asset_data = taker_asset_data
    order.exchange_address = exchange_address
    order.signature = w3_instance.eth.account.signHash(
        order.update().hash, private_key=private_key
    )["signature"].hex()
    assert_valid(order.to_json(), "/signedOrderSchema")
    return order

def sign_and_post_order(order):
    base_url = "http://localhost:3000/v2/"
    print(f"POSTING {order}")
    response = requests.post(
        "{}{}".format(base_url, "order"),
        json=order.to_json()
    )
    return response.json()

In [ ]:
from pydex_app.db_models import SignedOrder
from pprint import pprint

orderbook = get_orderbook(
    base_asset_data=VETH_ASSET_DATA,
    quote_asset_data=LONG_ASSET_DATA,
    full_set_asset_data=full_asset_set_data
)
# pprint(orderbook)

def print_order_book(orderbook):
    print("BIDS:---------")
    bids = [SignedOrder.from_json(order_json["order"]).update()
            for order_json in orderbook["bids"]["records"]]
    print("\n".join([str(bid) for bid in bids]))
    print("ASKS:---------")
    asks = [SignedOrder.from_json(order_json["order"]).update()
            for order_json in orderbook["asks"]["records"]]
    print("\n".join([str(bid) for bid in asks]))
    return bids, asks
    
bids, asks = print_order_book(orderbook)

In [ ]:
for side in ["BUY", "SELL"]:
    order = make_signed_order(
        asset="LONG",
        qty=0.0001,
        price=0.45,
        side=side,
    )
    print(json.dumps(order.to_json(include_hash=True)))
    print(sign_and_post_order(order))

In [ ]:
# from pydex_app.db_models import SignedOrder
# list(SignedOrder.query.filter_by(
#     maker_asset_data=VETH_ASSET_DATA))

In [ ]:
from pydex_app.order_watcher_client import OrderWatcherClient

owc = OrderWatcherClient(
    on_msg=print,
    on_error=print,
)
owc.run()

In [ ]:
for order in bids + asks:
    res = owc._rpc(method="ADD_ORDER", params={"signedOrder": order.to_json()})
    print(">>>>", res)

In [ ]:
res = owc._rpc(method="GET_STATS")
res

In [ ]:
for order in bids + asks:
    res = owc._rpc(method="REMOVE_ORDER", params={"orderHash": order.hash})
    print(">>>>", res)

In [ ]:
res = owc._rpc(method="GET_STATS")
res

In [ ]:
for side in ["BUY", "SELL"]:
    order = make_signed_order(
        asset="LONG",
        qty=0.0001,
        price=0.45,
        side=side,
    )
    print(json.dumps(order.to_json(include_hash=True)))
    print(sign_and_post_order(order))

In [ ]:
from web3 import Web3
from zero_ex.contract_artifacts import abi_by_name;


class TradeExecutor:
    def __init__(self, address, private_key, rpc_url, network_id, exchange_address):
        # Fee recipient address and private key
        self.address = address
        self.private_key = private_key
        # Initialize web3
        self.w3 = Web3(Web3.HTTPProvider(rpc_url, request_kwargs={"timeout": 60}))
        # Initalize zrx_exchange contract
        self.zrx_exchange = self.w3.eth.contract(
            address=exchange_address,
            abi=abi_by_name("exchange"),
        )
        # Initialize chainId
        self.network_id = network_id

    def execute_batch_fill_or_kill(
        self, orders, taker_asset_fill_amounts, signatures
    ):
        func_params = [orders, taker_asset_fill_amounts, signatures]
        function = zrx_exchange.functions.batchFillOrKill(*func_params)
        return self._build_and_send_tx(function)

    def _get_tx_params(self, value=0, gas=150000):
        # Get generic transcation parameters.
        return {
            "chainId": self.network_id,
            "from": self.address,
            "value": value,
            "gas": gas,
            "nonce": self.w3.eth.getTransactionCount(self.address)
        }

    def _build_and_send_tx(function):
        # Build and send a transaction to the zrx_exchange
        tx_params = _get_tx_params()
        transaction = function.buildTransaction(tx_params)
        signed_tx = self.w3.eth.account.signTransaction(
            transaction, private_key = self.private_key
        )
        return self.w3.eth.sendRawTransaction(signed_tx.sendRawTransaction)


In [ ]:
tx = TradeExecutor(
    address=Web3.toChecksumAddress(MY_ADDRESS),
    private_key=PRIVATE_KEY,
    rpc_url=WEB3_RPC_URL,
    network_id=NETWORK_ID,
    exchange_address=Web3.toChecksumAddress("0xbce0b5f6eb618c565c3e5f5cd69652bbc279f44e"),
)